In [1]:
import datetime
import os
import argparse
import torch

from rlpyt.samplers.collections import TrajInfo
from rlpyt.runners.minibatch_rl import MinibatchRlEval, MinibatchRl
from rlpyt.samplers.serial.sampler import SerialSampler
from rlpyt.utils.logging.context import logger_context

from dreamer_agent import DMCDreamerAgent
from A2C_algorithm import A2C_pok
from envs.dmc import DeepMindControl
from envs.time_limit import TimeLimit
from envs.action_repeat import ActionRepeat
from envs.normalize_actions import NormalizeActions
from envs.wrapper import make_wapper

#from rlpyt.algos.pg.a2c import A2C
from rlpyt.agents.pg.mujoco import MujocoFfAgent, MujocoLstmAgent

2024-01-14 01:08:37.425993: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-14 01:08:38.188895: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
def build_and_train(
    log_dir,
    game="cartpole_balance",
    run_ID=1,
    cuda_idx=None,
    eval=False,
    save_model="last",
    load_model_path=None,
):
    params = torch.load(load_model_path) if load_model_path else {}
    agent_state_dict = params.get("agent_state_dict")
    optimizer_state_dict = params.get("optimizer_state_dict")
    action_repeat = 2
    factory_method = make_wapper(
        DeepMindControl,
        [ActionRepeat, NormalizeActions, TimeLimit],
        [dict(amount=action_repeat), dict(), dict(duration=1000 / action_repeat)],
    )
    sampler = SerialSampler(
        EnvCls=factory_method,
        TrajInfoCls=TrajInfo,
        env_kwargs=dict(name=game),
        eval_env_kwargs=dict(name=game),
        batch_T=1,
        batch_B=1,
        max_decorrelation_steps=0,
        eval_n_envs=10,
        eval_max_steps=int(10e3),
        eval_max_trajectories=5,
    )
    algo = A2C_pok(initial_optim_state_dict=optimizer_state_dict)  # Run with defaults.
    agent = DMCDreamerAgent(
        train_noise=0.3,
        eval_noise=0,
        expl_type="additive_gaussian",
        expl_min=None,
        expl_decay=None,
        initial_model_state_dict=agent_state_dict,
    )
    runner_cls = MinibatchRlEval if eval else MinibatchRl
    runner = runner_cls(
        algo=algo,
        agent=agent,
        sampler=sampler,
        n_steps=1000000,
        log_interval_steps=1e3,
        affinity=dict(cuda_idx=cuda_idx),
    )
    config = dict(game=game)
    name = "dreamer_" + game
    with logger_context(
        log_dir,
        run_ID,
        name,
        config,
        snapshot_mode=save_model,
        override_prefix=True,
        use_summary_writer=True,
    ):
        runner.train()


In [3]:
log_dir = os.path.abspath('/home/eddy/Projects/RL_project/logs_A2C')

build_and_train(
        log_dir,
        game="walker_walk",
        run_ID=1,
        cuda_idx=0,
        eval=False,
        save_model="last",
        load_model_path=False,
    )

2024-01-14 01:08:39.280653  | dreamer_walker_walk_1 Runner  master CPU affinity: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11].
2024-01-14 01:08:39.281155  | dreamer_walker_walk_1 Runner  master Torch threads: 6.


using seed 7753
2024-01-14 01:08:40.760598  | dreamer_walker_walk_1 Sampler decorrelating envs, max steps: 0
2024-01-14 01:08:40.762069  | dreamer_walker_walk_1 Serial Sampler initialized.
2024-01-14 01:08:40.762446  | dreamer_walker_walk_1 Running 1000000 iterations of minibatch RL.
2024-01-14 01:08:42.192891  | dreamer_walker_walk_1 Initialized agent model on device: cuda:0.
DAI CHE SIAMO VICINIIIIIIII
size passato a SequenceNStepReturnBuffer = 300000
2024-01-14 01:08:42.195224  | dreamer_walker_walk_1 Optimizing over 1000 iterations.


/home/eddy/.local/lib/python3.10/site-packages/torch/optim/adam.py:33: UserWarning: optimizer contains a parameter group with duplicate parameters; in future, this will cause an error; see github.com/pytorch/pytorch/issues/40967 for more information
  super().__init__(params, defaults)


2024-01-14 01:08:51.874336  | dreamer_walker_walk_1 itr #999 saving snapshot...
2024-01-14 01:08:51.919743  | dreamer_walker_walk_1 itr #999 saved
2024-01-14 01:08:51.930604  | -----------------------------  ----------
2024-01-14 01:08:51.931090  | Diagnostics/NewCompletedTrajs     2
2024-01-14 01:08:51.931411  | Diagnostics/StepsInTrajWindow  1000
2024-01-14 01:08:51.931832  | Diagnostics/Iteration           999
2024-01-14 01:08:51.932208  | Diagnostics/CumTime (s)           9.72551
2024-01-14 01:08:51.933008  | Diagnostics/CumSteps           1000
2024-01-14 01:08:51.933383  | Diagnostics/CumCompletedTrajs     2
2024-01-14 01:08:51.933883  | Diagnostics/CumUpdates            0
2024-01-14 01:08:51.934574  | Diagnostics/StepsPerSecond      102.822
2024-01-14 01:08:51.935351  | Diagnostics/UpdatesPerSecond      0
2024-01-14 01:08:51.935715  | Diagnostics/ReplayRatio           0
2024-01-14 01:08:51.936348  | Diagnostics/CumReplayRatio        0
2024-01-14 01:08:51.936734  | Length/Average 

A2C: 100%|██████████| 100/100 [01:48<00:00,  1.08s/it]


...

2024-01-14 08:31:34.980395  | dreamer_walker_walk_1 itr #214999 saving snapshot...
2024-01-14 08:31:35.043386  | dreamer_walker_walk_1 itr #214999 saved
2024-01-14 08:31:35.053002  | -----------------------------  ----------------
2024-01-14 08:31:35.053715  | Diagnostics/NewCompletedTrajs       2
2024-01-14 08:31:35.054079  | Diagnostics/StepsInTrajWindow   50000
2024-01-14 08:31:35.054480  | Diagnostics/Iteration          214999
2024-01-14 08:31:35.055208  | Diagnostics/CumTime (s)         26572.8
2024-01-14 08:31:35.055587  | Diagnostics/CumSteps           215000
2024-01-14 08:31:35.055952  | Diagnostics/CumCompletedTrajs     430
2024-01-14 08:31:35.056400  | Diagnostics/CumUpdates              0
2024-01-14 08:31:35.056744  | Diagnostics/StepsPerSecond          7.82914
2024-01-14 08:31:35.057071  | Diagnostics/UpdatesPerSecond        0
2024-01-14 08:31:35.057524  | Diagnostics/ReplayRatio             0
2024-01-14 08:31:35.057893  | Diagnostics/CumReplayRatio          0
2024-01-14 08

A2C: 100%|██████████| 100/100 [01:54<00:00,  1.15s/it]


2024-01-14 08:33:39.548782  | dreamer_walker_walk_1 itr #215999 saving snapshot...
2024-01-14 08:33:39.618813  | dreamer_walker_walk_1 itr #215999 saved
2024-01-14 08:33:39.627282  | -----------------------------  ----------------
2024-01-14 08:33:39.627881  | Diagnostics/NewCompletedTrajs       2
2024-01-14 08:33:39.628207  | Diagnostics/StepsInTrajWindow   50000
2024-01-14 08:33:39.628624  | Diagnostics/Iteration          215999
2024-01-14 08:33:39.629067  | Diagnostics/CumTime (s)         26697.4
2024-01-14 08:33:39.629502  | Diagnostics/CumSteps           216000
2024-01-14 08:33:39.630205  | Diagnostics/CumCompletedTrajs     432
2024-01-14 08:33:39.630487  | Diagnostics/CumUpdates              0
2024-01-14 08:33:39.630845  | Diagnostics/StepsPerSecond          8.02729
2024-01-14 08:33:39.631459  | Diagnostics/UpdatesPerSecond        0
2024-01-14 08:33:39.631879  | Diagnostics/ReplayRatio             0
2024-01-14 08:33:39.632292  | Diagnostics/CumReplayRatio          0
2024-01-14 08

A2C: 100%|██████████| 100/100 [02:03<00:00,  1.23s/it]


KeyboardInterrupt: 